In [2]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
import keras

from settings import *

from utils.general import clean_session, plot_model, split_dataset, scale_dataset, shape_dataset, print_dataset_info
from utils.models.ocr import save_model

2024-03-23 19:00:33.970825: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-23 19:00:34.256789: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-23 19:00:38.407545: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import Datasets.OCR.EnglishFnt.loader as EnglishFnt

In [4]:
def create_model():
    model = Sequential()
    # Signle convolution to extract features

    # CNN
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',input_shape=INPUT_SHAPE))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))

    # Pooling
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))

    # Drop some data points randomly (mainly for caligraphy reasons)
    model.add(Dropout(0.25))

    # Flattening before feeding to FCNN
    model.add(Flatten())

    # FCNN
    # use softmax since we are doing classification (1 node for each possible output)
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(COMBINED_CHARS), activation='softmax'))

    model.summary()

    return model


In [5]:
def train_model(model, learning_rate, batch_size, epochs):
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss=categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])

    EX_data, EY_data = EnglishFnt.load_dataset()
    X_data = EX_data
    Y_data = EY_data

    X_train, X_test, Y_train, Y_test = split_dataset(X_data, Y_data, test_percent=TEST_PERCENT, random_state=RANDOM_STATE)
    X_train, X_test = shape_dataset(X_train, X_test, input_shape=INPUT_SHAPE)
    X_train, X_test = scale_dataset(X_train, X_test)

    print_dataset_info(X_data, X_train, Y_train, X_test, Y_test)

    history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_split=VALIDATION_PERCENT)

    return history

In [6]:
model = create_model()
history = train_model(model, learning_rate=0.001, batch_size=32, epochs=15)
plot_model(history)
save_model(model, 1)

/home/evangelospro/Downloads/Final-Project/.venv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
2024-03-23 19:00:47.978984: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-23 19:00:48.108295: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 98, 48, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 96, 46, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 94, 44, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 47, 22, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 47, 22, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 132352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    16,941,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 35)             │         4,515 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,038,371 (65.00 MB)

 Trainable params: 17,038,371 (65.00 MB)

...

2071/2071 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.7031 - loss: 1.0900

2024-03-23 19:04:44.610772: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 373.57MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-03-23 19:04:44.875347: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 239.88MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


2071/2071 ━━━━━━━━━━━━━━━━━━━━ 162s 62ms/step - accuracy: 0.7032 - loss: 1.0897 - val_accuracy: 0.9664 - val_loss: 0.1418
Epoch 2/15


2024-03-23 19:04:58.048192: W external/local_tsl/tsl/framework/bfc_allocator.cc:487] Allocator (GPU_0_bfc) ran out of memory trying to allocate 339.61MiB (rounded to 356112384)requested by op 
2024-03-23 19:04:58.048432: I external/local_tsl/tsl/framework/bfc_allocator.cc:1044] BFCAllocator dump for GPU_0_bfc
2024-03-23 19:04:58.048497: I external/local_tsl/tsl/framework/bfc_allocator.cc:1051] Bin (256): 	Total Chunks: 2126, Chunks in use: 2126. 531.5KiB allocated for chunks. 531.5KiB in use in bin. 9.8KiB client-requested in use in bin.
2024-03-23 19:04:58.048546: I external/local_tsl/tsl/framework/bfc_allocator.cc:1051] Bin (512): 	Total Chunks: 7, Chunks in use: 7. 3.5KiB allocated for chunks. 3.5KiB in use in bin. 3.3KiB client-requested in use in bin.
2024-03-23 19:04:58.048590: I external/local_tsl/tsl/framework/bfc_allocator.cc:1051] Bin (1024): 	Total Chunks: 4, Chunks in use: 4. 5.0KiB allocated for chunks. 5.0KiB in use in bin. 4.4KiB client-requested in use in bin.
2024-03-2

ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/evangelospro/Downloads/Final-Project/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/evangelospro/Downloads/Final-Project/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/evangelospro/Downloads/Final-Project/.venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/evangelospro/Downloads/Final-Project/.venv/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _